In [1]:
# rsa algorithm
from __future__ import unicode_literals
from math import sqrt
import random
import sympy
import binascii
from random import randint as rand
from random import getrandbits
from random import randrange

def gcd(a, b):
    if a == 0:
        return b
    else:
        return gcd(b%a, a)

def extended(a, b): 
    if a == 0 : 
        return b, 0, 1
    gcd, x1, y1 = extended(b%a, a) 

    x = y1 - (b//a) * x1 
    y = x1 
    
    return (gcd,x,y)

def rabin_miller(n, k=128):
    if n == 2 or n == 3:                 # if no is either 2 or 3 return true
        return True
    if n <= 1 or n % 2 == 0:          # no should be greater than 1 and should not be even as prime nos are odd's
        return False
    s = 0
    r = n - 1
    while r & 1 == 0:                    # calculate values of r and s so that (n-1)=r*2**s is satisfied
        s += 1
        r //= 2

    for _ in range(k):                    # no of test to do and it holds result of last execution
        a = randrange(2, n - 1)
        x = pow(a, r, n)                    # modulus by n to get x in range of 2 to n ( a** r mod n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:      # derive  value for j such that below condition is true
                x = pow(x, 2, n)
                if x == 1:                     # check for second condition (a**(2**j)r!=-1 mod n)
                    return False
                j += 1
            if x != n - 1: 
                return False
    return True

def inverse_modulo(a, m):
    g, x, y = extended(a, m)
    if g != 1:
        return None  
    else:
        return x % m



def prime(length=1024):
    p = 4
    while not rabin_miller(p, 128):            #while not true
        a = '1'+'0'*(length-1)
        b = '1'*length
        p = random.randrange(int(a,2),int(b,2))
     # p = random.randrange(2**(length-1) + 1, 2**(length) - 1)

    return p


def generate_large_no(keysize):
    p = prime(keysize)
    q = prime(keysize)
    n = p * q
    phi = (p-1)*(q-1)//gcd(p-1, q-1)
    e = sympy.randprime(1,phi)
    d = inverse_modulo(e,phi)
    if e != d:
        return ((e, n), (d, n))  


def encrypt(plain_text, package):
    e, n = package
    if plain_text > n:
        print('Message is too large for key to handle')
        return 1
    msg_ciphertext = pow(plain_text, e, n)
    return msg_ciphertext


def decrypt(msg_ciphertext, package):
    d, n = package
    msg_plaintext = pow(msg_ciphertext, d, n)
    #return msg_plaintext
    return binascii.unhexlify(hex(msg_plaintext)[2:]).decode()             # un hex the msg and return



In [ ]:
# msg = "Hello this is an encrypted message"
# spublic,sprivate = generate_large_no(1024)
# cpublic,cprivate = generate_large_no(1024)

# print("The original message is: ",msg)

# message = str.encode(msg)
# # print(message)
# hex_data   = binascii.hexlify(message)         # convert string to hex format
# plain_text = int(hex_data, 16)
# en_msg = encrypt(plain_text,spublic)
# # print(en_msg)
# print("\nThe message from client is encrypted using server public key pair: ",str(en_msg).encode())

# #send_msg = str(en_msg).encode()
# # res_msg = en_msg
# # print("received msg: ",res_msg, "\n")
# plain_msg = decrypt(en_msg,sprivate)
# # print(str(plain_msg))
# print("\nThe decrypted message from client is: ",str(plain_msg))